<a href="https://colab.research.google.com/github/diegocgribeiro/previsao_brasileirao24/blob/main/modeloregreessaobr24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Construção de Base de Dados e Previsão Brasileirão!**

In [ ]:
pip install pandas sqlalchemy pymysql
pip install mysql-connector-python

**Função para envio ao Banco de Dados!**

Obs: Informações do banco de dados foram ocultadas.

In [ ]:
def enviar_para_mysql(df, tabela):
    try:
        conexao = mysql.connector.connect(
        host='*****.mysql.uhserver.com',
        user='data_user',
        password='*******',
        database='******'
)
        cursor = conexao.cursor()
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {tabela} (
            Posição INT,
            Times VARCHAR(100),
            Pontos FLOAT,
            Jogos INT,
            Jogos_Casa INT,
            Jogos_Fora INT,
            Vitorias INT,
            Vitorias_Casa INT,
            Vitorias_Fora INT,
            Derrotas INT,
            Derrotas_Casa INT,
            Derrotas_Fora INT,
            Empates INT,
            Empates_Casa INT,
            Empates_Fora INT,
            Gols_Feitos INT,
            Gols_Feitos_Casa INT,
            Gols_Feitos_Fora INT,
            Gols_Tomados INT,
            Gols_Tomados_Casa INT,
            Gols_Tomados_Fora INT,
            Saldo_de_Gols INT,
            Saldo_de_Gols_Casa INT,
            Saldo_de_Gols_Fora INT,
            Aproveitamento FLOAT,
            Aproveitamento_Casa FLOAT,
            Aproveitamento_Fora FLOAT,
            Rodada INT
        )
        """)

        # Insert data from classificacoes_df into the table
        for index, row in classificacoes_df.iterrows():
            cursor.execute(f"""
            INSERT INTO {tabela} (Posição, Times, Pontos, Jogos, Jogos_Casa, Jogos_Fora, Vitorias, Vitorias_Casa, Vitorias_Fora,
                                  Derrotas, Derrotas_Casa, Derrotas_Fora, Empates, Empates_Casa, Empates_Fora,
                                  Gols_Feitos, Gols_Feitos_Casa, Gols_Feitos_Fora, Gols_Tomados, Gols_Tomados_Casa,
                                  Gols_Tomados_Fora, Saldo_de_Gols, Saldo_de_Gols_Casa, Saldo_de_Gols_Fora,
                                  Aproveitamento, Aproveitamento_Casa, Aproveitamento_Fora, Rodada)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(row))

        # Commit changes
        conexao.commit()

        print(f'Dados inseridos com sucesso na tabela {tabela}')

    except mysql.connector.Error as err:
        print({err})

    finally:
        if conexao.is_connected():
            cursor.close()
            conexao.close()

**WEB SCRAPING**

Cédula que usando a função BeautifulSoup, faz web scraping, obtendo os dados do site e transformando em DataFrame

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
times = [
    "Internacional", "Bahia", "Criciúma", "Juventude", "Fluminense",
    "Red Bull Bragantino", "São Paulo", "Fortaleza", "Atlético-GO",
    "Flamengo", "Vasco", "Grêmio", "Corinthians", "Atlético-MG",
    "Athletico-PR", "Cuiabá", "Cruzeiro", "Botafogo", "Vitória", "Palmeiras", "Inter", "Bragantino"
]

url = "https://olympics.com/pt/noticias/campeonato-brasileiro-2024-todos-resultados"

# Adicionar um cabeçalho User-Agent para simular um navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find_all('li')
    data = []
    rodadas = []
    for result in results:
        result_text = result.get_text().strip()
        if any(time in result_text for time in times):
            data.append({"Resultado": result_text})
else:
    print(f"Erro: Não foi possível acessar a página. Status Code: {response.status_code}")

df = pd.DataFrame(data)

**Formatação e manipulação dos dados**

Feito a separação em 4 colunas (Time A, Gols Time A, Time B e Gols Time B) e feito a conversão para arquivo CSV

In [ ]:
for i in range(1, 39):
    for j in range(10):
      rodadas.append(i)

df['Rodada'] = rodadas
df = df[~df['Resultado'].str.contains(r'A definir|adiado', case=False, na=False)]
df['Resultado'] = df['Resultado'].replace({'Atlético-MG': 'AtléticoMG', 'Atlético-GO': 'AtléticoGO', 'Athletico-PR': 'AthleticoPR', 'São Paulo': 'SPFC', 'Red Bull Bragantino': 'Bragantino', 'Internacional': 'Inter'}, regex=True)
df[['Time A']] = df['Resultado'].str.extract(r'(\w+)')
df[['Time B']] = df['Resultado'].str.extract(r'(\w+)$')
df[['Gols Time A']] = df['Resultado'].str.extract(r'(\d+)')
df[['Gols Time B']] = df['Resultado'].str.extract(r'x (\d+)')
times_a = set(df['Time A'].unique())
times_b = set(df['Time B'].unique())

df.to_csv('resultados.csv', index=False)

**Construção da tabela completa**

Tendo os resultados e a quantidade de gols, foi possivel montar uma tabela completa do Brasileirão por Rodada

In [ ]:
import pandas as pd
df = pd.read_csv('resultados.csv')
futuros_jogos = df[df['Gols Time A'].isna()]
df = df.dropna(subset=['Gols Time A'])
classificacoes_por_rodada = []
for rodada in range(1, df['Rodada'].max()+1):
  df_rodada = df[df['Rodada'] <= rodada]
  classificacao = pd.DataFrame()
  classificacao['Posição'] = 0
  classificacao['Times'] = df['Time A'].unique()
  classificacao['Pontos'] = 0
  classificacao['Jogos'] = 0
  classificacao['Jogos Casa'] = 0
  classificacao['Jogos Fora'] = 0
  classificacao['Vitorias'] = 0
  classificacao['Vitorias Casa'] = 0
  classificacao['Vitorias Fora'] = 0
  classificacao['Derrotas'] = 0
  classificacao['Derrotas Casa'] = 0
  classificacao['Derrotas Fora'] = 0
  classificacao['Empates'] = 0
  classificacao['Empates Casa'] = 0
  classificacao['Empates Fora'] = 0
  classificacao['Gols Feitos'] = 0
  classificacao['Gols Feitos Casa'] = 0
  classificacao['Gols Feitos Fora'] = 0
  classificacao['Gols Tomados'] = 0
  classificacao['Gols Tomados Casa'] = 0
  classificacao['Gols Tomados Fora'] = 0
  classificacao['Saldo de Gols'] = 0
  classificacao['Saldo de Gols Casa'] = 0
  classificacao['Saldo de Gols Fora'] = 0
  classificacao['Aproveitamento'] = 0
  classificacao['Aproveitamento Casa'] = 0
  classificacao['Aproveitamento Fora'] = 0
  for index, row in df_rodada.iterrows():
    if row['Gols Time A'] > row['Gols Time B']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Vitorias Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Derrotas Fora'] += 1
    elif row['Gols Time B'] > row['Gols Time A']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Derrotas Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Vitorias Fora'] += 1
    else:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Empates Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Empates Fora'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Feitos Casa'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Feitos Fora'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Tomados Casa'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Tomados Fora'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Jogos Casa'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Jogos Fora'] += 1
  principais =['Jogos','Vitorias', 'Derrotas', 'Empates','Jogos', 'Gols Feitos', 'Gols Tomados']
  for i in principais:
    classificacao[i] = classificacao[f'{i} Casa'] + classificacao[f'{i} Fora']
  #classificacao['Jogos'] = classificacao['Jogos Casa'] + classificacao['Jogos Fora']
  #classificacao['Vitorias'] = classificacao['Vitorias Casa'] + classificacao['Vitorias Fora']
  classificacao['Saldo de Gols Casa'] = classificacao['Gols Feitos Casa'] - classificacao['Gols Tomados Casa']
  classificacao['Saldo de Gols Fora'] = classificacao['Gols Feitos Fora'] - classificacao['Gols Tomados Fora']
  classificacao['Pontos'] = classificacao['Vitorias'] * 3 + classificacao['Empates'] * 1
  classificacao['Aproveitamento'] = classificacao['Pontos']/(classificacao['Jogos']*3)
  classificacao['Aproveitamento Casa'] = (classificacao['Vitorias Casa']*3 +classificacao['Empates Casa']*1)/(classificacao['Jogos Casa']*3)
  classificacao['Aproveitamento Fora'] = (classificacao['Vitorias Fora']*3 +classificacao['Empates Fora']*1)/(classificacao['Jogos Fora']*3)
  classificacao['Saldo de Gols'] = classificacao['Gols Feitos'] - classificacao['Gols Tomados']
  classificacao = classificacao.sort_values(by=['Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
  classificacao['Posição'] = list(range(1, 21))
  classificacao['Rodada'] = rodada
  classificacoes_por_rodada.append(classificacao)
classificacoes_df = pd.concat(classificacoes_por_rodada)
classificacoes_df = classificacoes_df.sort_values(by=['Rodada', 'Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
classificacoes_df.to_excel('tabela_brasileiraoate30.xlsx')
minhabase = classificacoes_df

**Enviando para o MYSQL**

Subindo a base de dados para SQL, usando a função enviar_para_mysql

In [ ]:
enviar_para_mysql(minhabase, 'tabela_minhabase3')

**Prevendo futuros resultados**

Usando apenas médias de gols feitos, gols tomados e entre outros (não teve o uso de regressões lineares, modelo logisticos e entre outros), foi possivel calcular a quantidade de gols que data time irá fazer por jogo

**Adicionei alguns fatores:**

Fator grandeza - Levando em considerações titulos nacioanais, foi separado os 20 times, em 3 grupos, 'Pequeno', 'Médio' e 'Grande'

Posição atual - Foi levado em consideração a posição atual que o time se encontra no campeoanto


Ultimos 5 jogos - Foi levado em consideração a campanha feito nos ultimos 5 jogos do Brasileirão

Obs: Fator que optei por não colocar, é se o time está jogando outra competição visto que pode ou não interferir.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('resultados.csv')
futuros_jogos = df[df['Gols Time A'].isna()]

grande = [
    "Palmeiras", "Flamengo", "SPFC", "Inter",
    "Cruzeiro", "Grêmio", "Fluminense", "Corinthians"
]

medio = [
    "Botafogo", "Fortaleza", "Bahia",
    "AtléticoMG", "Vasco", "AthleticoPR"
]

pequeno = [
    "Criciúma", "Bragantino", "Juventude",
    "Vitória", "Cuiabá", "AtléticoGO"
]

for index, row in futuros_jogos.iterrows():

  gols_feitos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Feitos Casa'].values[0]
  jogos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Casa'].values[0]
  gols_tomados_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Tomados Fora'].values[0]
  jogos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Fora'].values[0]
  posicao_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Posição'].values[0]
  posicao_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Posição'].values[0]
  pontos5jogosatras = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()-5), 'Pontos'].values[0]
  pontosagora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Pontos'].values[0]
  if row['Time A'] in grande:
    fator_grandeza = 1.1
  elif row['Time A'] in medio:
    fator_grandeza = 1
  else:
    fator_grandeza = 0.9
  media_gols_casa_feitos = gols_feitos_casa/jogos_casa
  media_gols_fora_tomados = gols_tomados_fora/jogos_fora
  fator5jogos = 1+((pontosagora - pontos5jogosatras)/50)
  gols_esperadosA = ((((media_gols_casa_feitos*media_gols_fora_tomados))*(1+(((20-posicao_casa)*2))/100)*fator_grandeza)*fator5jogos)*0.6

  gols_feitos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Feitos Fora'].values[0]
  jogos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Fora'].values[0]
  gols_tomados_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Tomados Casa'].values[0]
  jogos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Casa'].values[0]
  pontos5jogosatras = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()-5), 'Pontos'].values[0]
  pontosagora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Pontos'].values[0]
  if row['Time B'] in grande:
    fator_grandeza = 1.1
  elif row['Time B'] in medio:
    fator_grandeza = 1
  else:
    fator_grandeza = 0.9
  fator5jogos = 1+((pontosagora - pontos5jogosatras)/50)
  media_gols_fora_feitos = gols_feitos_fora/jogos_fora
  media_gols_casa_tomados = gols_tomados_casa/jogos_casa
  gols_esperadosB = ((((media_gols_fora_feitos*media_gols_casa_tomados))*(1+(((20-posicao_fora)*2))/100)*fator_grandeza)*fator5jogos)*0.6
  futuros_jogos.loc[(futuros_jogos['Time A'] == row['Time A']) & (futuros_jogos['Time B'] == row['Time B']), 'Gols Time A'] = gols_esperadosA.round(0)
  futuros_jogos.loc[(futuros_jogos['Time A'] == row['Time A']) & (futuros_jogos['Time B'] == row['Time B']), 'Gols Time B'] = gols_esperadosB.round(0)
futuros_jogos.to_excel('jogos_futuros.xlsx')

**Transformando em DataFrame**

In [ ]:
import pandas as pd
import numpy as np

df = df.dropna(subset=['Gols Time A'])
df = pd.concat([df, futuros_jogos], ignore_index=True)
classificacoes_por_rodada = []
for rodada in range(1, df['Rodada'].max()+1):
  df_rodada = df[df['Rodada'] <= rodada]
  classificacao = pd.DataFrame()
  classificacao['Posição'] = 0
  classificacao['Times'] = df['Time A'].unique()
  classificacao['Pontos'] = 0
  classificacao['Jogos'] = 0
  classificacao['Jogos Casa'] = 0
  classificacao['Jogos Fora'] = 0
  classificacao['Vitorias'] = 0
  classificacao['Vitorias Casa'] = 0
  classificacao['Vitorias Fora'] = 0
  classificacao['Derrotas'] = 0
  classificacao['Derrotas Casa'] = 0
  classificacao['Derrotas Fora'] = 0
  classificacao['Empates'] = 0
  classificacao['Empates Casa'] = 0
  classificacao['Empates Fora'] = 0
  classificacao['Gols Feitos'] = 0
  classificacao['Gols Feitos Casa'] = 0
  classificacao['Gols Feitos Fora'] = 0
  classificacao['Gols Tomados'] = 0
  classificacao['Gols Tomados Casa'] = 0
  classificacao['Gols Tomados Fora'] = 0
  classificacao['Saldo de Gols'] = 0
  classificacao['Saldo de Gols Casa'] = 0
  classificacao['Saldo de Gols Fora'] = 0
  classificacao['Aproveitamento'] = 0
  classificacao['Aproveitamento Casa'] = 0
  classificacao['Aproveitamento Fora'] = 0
  for index, row in df_rodada.iterrows():
    if row['Gols Time A'] > row['Gols Time B']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Vitorias Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Derrotas Fora'] += 1
    elif row['Gols Time B'] > row['Gols Time A']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Derrotas Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Vitorias Fora'] += 1
    else:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Empates Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Empates Fora'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Feitos Casa'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Feitos Fora'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Tomados Casa'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Tomados Fora'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Jogos Casa'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Jogos Fora'] += 1
  principais =['Jogos','Vitorias', 'Derrotas', 'Empates','Jogos', 'Gols Feitos', 'Gols Tomados']
  for i in principais:
    classificacao[i] = classificacao[f'{i} Casa'] + classificacao[f'{i} Fora']
  classificacao['Jogos'] = classificacao['Jogos Casa'] + classificacao['Jogos Fora']
  classificacao['Vitorias'] = classificacao['Vitorias Casa'] + classificacao['Vitorias Fora']
  classificacao['Saldo de Gols Casa'] = classificacao['Gols Feitos Casa'] - classificacao['Gols Tomados Casa']
  classificacao['Saldo de Gols Fora'] = classificacao['Gols Feitos Fora'] - classificacao['Gols Tomados Fora']
  classificacao['Pontos'] = classificacao['Vitorias'] * 3 + classificacao['Empates'] * 1
  classificacao['Aproveitamento'] = classificacao['Pontos']/(classificacao['Jogos']*3)
  classificacao['Aproveitamento Casa'] = (classificacao['Vitorias Casa']*3 +classificacao['Empates Casa']*1)/(classificacao['Jogos Casa']*3)
  classificacao['Aproveitamento Fora'] = (classificacao['Vitorias Fora']*3 +classificacao['Empates Fora']*1)/(classificacao['Jogos Fora']*3)
  classificacao['Saldo de Gols'] = classificacao['Gols Feitos'] - classificacao['Gols Tomados']
  classificacao = classificacao.sort_values(by=['Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
  classificacao['Posição'] = list(range(1, 21))
  classificacao['Rodada'] = rodada
  classificacoes_por_rodada.append(classificacao)
classificacoes_df = pd.concat(classificacoes_por_rodada)
classificacoes_df = classificacoes_df.sort_values(by=['Rodada', 'Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
classificacoes_df.to_excel('tabela_brasileiraocompleta.xlsx')
classificacoes_df

**Enviando a tabela para MYSQL**

In [ ]:
enviar_para_mysql(classificacoes_df, 'tabela_minhabaseprevisao3')